[Chaining, or Why You Should Stop Returning Void](http://jamie-wong.com/2012/02/01/stop-returning-void/)

> The unfortunate reality of chaining in statically typed languages is that it doesn’t play nice with inheritance.

Or is it?

[Method chaining + inheritance don't play well together?](https://stackoverflow.com/questions/551263/method-chaining-inheritance-dont-play-well-together)


In [1]:
#include <iostream>
[]()
{
auto run_add_err = []()
{
struct Point{
  Point():
    x(0),
    y(0){}
  Point(double _x, double _y):
    x(_x),
    y(_y){}
  double x,y;
};
struct Image{
  void setAngle(double angle){}  
};
// member data omitted for brevity

// assume that "setAngle" needs to be implemented separately
// in Label and Image, and that Button does need to inherit
// Label, rather than, say, contain one (etc)
struct Widget {
    Widget(){}
    Widget& move(Point newPos) { pos = newPos; return *this; }
    Point pos;
};

struct Label : Widget {
    Label(){}
    Label& setText(std::string const& newText) { text = newText; return *this; }
    Label& setAngle(double newAngle) { angle = newAngle; return *this; }
    std::string text;
    double angle;
};

    
struct Button : Label {
    Button(){}
    Image backgroundImage;
    Button& setAngle(double newAngle) {
        backgroundImage.setAngle(newAngle);
        Label::setAngle(newAngle);
        return *this;
    }
};

Button btn;

// oops: Widget::setText doesn't exist
btn.move(Point(0,0)).setText("Hey");

// oops: calling Label::setAngle rather than Button::setAngle
btn.setText("Boo").setAngle(.5); 

};
run_add_err();
}();

input_line_8:46:22: error: no member named 'setText' in 'Widget'
btn.move(Point(0,0)).setText("Hey");
~~~~~~~~~~~~~~~~~~~~ ^


Interpreter Error: 

In [3]:
[]()
{
auto run_add_err = []()
{

template <typename Q, typename T>
class Default {
public:
    typedef Q type;
};

template <typename T>
class Default<void, T> {
public:
    typedef T type;
};

class Point{
public:
  Point():
    x(0),
    y(0){}
  Point(double _x, double _y):
    x(_x),
    y(_y){}
  double x,y;
};
class Image{
public:
  void setAngle(double angle){}
};


template<class T>
class Widget {
public:
    typedef typename Default<T, Widget<void> >::type type;
    Widget(){}
    type& move(Point newPos) { pos = newPos; return *this; }
    Point pos;
};

template<class T=void>
class Label : Widget<Label<T> > {
public:
     typedef typename Default<T, Widget<Label<T> > >::type type;
    Label(){}
    type& setText(std::string const& newText) { text = newText; return *this; }
    type& setAngle(double newAngle) { angle = newAngle; return *this; }
    std::string text;
    double angle;
};

template<class T=void>
class Button : Label<Button<T> > {
public:
    typedef typename Default<T, Label<Button<T> > >::type type;

    Button(){}
    Image backgroundImage;
    type& setAngle(double newAngle) {
        backgroundImage.setAngle(newAngle);
        Label<Button<T> >::setAngle(newAngle);
        return *this;
    }
};

Button<> btn;

btn.move(Point(0,0)).setText("Hey");

// // oops: calling Label::setAngle rather than Button::setAngle
// btn.setText("Boo").setAngle(.5); 

};
run_add_err();
}();

input_line_10:6:1: error: expected expression
template <typename Q, typename T>
^
input_line_10:11:1: error: expected expression
template <typename T>
^
input_line_10:30:1: error: expected expression
template<class T>
^
input_line_10:38:1: error: expected expression
template<class T=void>
^
input_line_10:48:1: error: expected expression
template<class T=void>
^
input_line_10:60:1: error: use of undeclared identifier 'Button'
Button<> btn;
^
input_line_10:60:8: error: expected expression
Button<> btn;
       ^
input_line_10:60:10: error: use of undeclared identifier 'btn'
Button<> btn;
         ^


Interpreter Error: 